In [2]:
#https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-Xception-a81a4a28084b
# Running the version as 1.x is optional, without that first line it will run the last version of tensorflow for Colab.

%tensorflow_version 1.x
import tensorflow as tf
import tensorflow.keras as K




TensorFlow 1.x selected.


In [3]:
# Load data
# Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples 
# and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. 
# We can take advantage of the fact that these categories and a lot more are into the Imagenet collection.

#tf.keras.datasets.cifar10.load_data()
#tf.keras.datasets.cifar100.load_data(label_mode="coarse")
tf.keras.datasets.cifar100.load_data()


169009152/169001437 [==============================] - 3s 0us/step


((array([[[[255, 255, 255],
           [255, 255, 255],
           [255, 255, 255],
           ...,
           [195, 205, 193],
           [212, 224, 204],
           [182, 194, 167]],
  
          [[255, 255, 255],
           [254, 254, 254],
           [254, 254, 254],
           ...,
           [170, 176, 150],
           [161, 168, 130],
           [146, 154, 113]],
  
          [[255, 255, 255],
           [254, 254, 254],
           [255, 255, 255],
           ...,
           [189, 199, 169],
           [166, 178, 130],
           [121, 133,  87]],
  
          ...,
  
          [[148, 185,  79],
           [142, 182,  57],
           [140, 179,  60],
           ...,
           [ 30,  17,   1],
           [ 65,  62,  15],
           [ 76,  77,  20]],
  
          [[122, 157,  66],
           [120, 155,  58],
           [126, 160,  71],
           ...,
           [ 22,  16,   3],
           [ 97, 112,  56],
           [141, 161,  87]],
  
          [[ 87, 122,  41],
           [ 8

In [4]:
# Preprocess data function
# Now that the data is loaded, we are going to build a preprocess function for the data. 
# We have X as a numpy array of shape (m, 32, 32, 3) where m is the number of images, 
# 32 and 32 the dimensions, and 3 is because we use color images (RGB). 
# We have a set of X for training and a set of X for validation. 
# Y is a numpy array of shape (m, ) that we want to be our labels. 
# Since we work with 10 different categories, we make use of one-hot encoding with a 
# function of Keras that makes our Y into a shape of (m, 10). That also applies for the validation.

def preprocess_data(X,Y):
  X_p = K.applications.xception.preprocess_input(X)
  Y_p = K.utils.to_categorical(Y,100)
  return X_p, Y_p

In [5]:
# load and split data
# The data, split between train and test sets:

(x_train, y_train), (x_test, y_test) = K.datasets.cifar100.load_data()
img_rows, img_cols = 32, 32
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)


x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [6]:
# Preprocess data
# Next, we are going to call our function with the parameters loaded from the Fashion Mnist database.

x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 100)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 100)


In [7]:
# Using weights of a trained neural network
# A pretrained model from the Keras Applications has the advantage of allowing you to use weights that
# are already calibrated to make predictions. In this case, we use the weights from Imagenet 
# and the network is a Xception. The option include_top=False allows feature extraction by removing 
# the last dense layers. This let us control the output and input of the model.

input_t = K.Input(shape=(32,32,3))
Xception_Model = K.applications.Xception(include_top=False,
                                    weights="imagenet",
                                    input_tensor=input_t)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
83689472/83683744 [==============================] - 1s 0us/step


In [8]:
# In this case, we ‘freeze’ all layers except for the last block of the Xception.

for layer in Xception_Model.layers[:]:
  layer.trainable=False
  

In [9]:
# We can check that we did it correctly with:
# False means that the layer is ‘freezed’ or is not trainable and 
# True that when we run our model, the weights are going to be adjusted.

for i, layer in enumerate(Xception_Model.layers):
  print(i,layer.name,"-",layer.trainable)
  

0 input_1 - False
1 block1_conv1 - False
2 block1_conv1_bn - False
3 block1_conv1_act - False
4 block1_conv2 - False
5 block1_conv2_bn - False
6 block1_conv2_act - False
7 block2_sepconv1 - False
8 block2_sepconv1_bn - False
9 block2_sepconv2_act - False
10 block2_sepconv2 - False
11 block2_sepconv2_bn - False
12 conv2d - False
13 block2_pool - False
14 batch_normalization - False
15 add - False
16 block3_sepconv1_act - False
17 block3_sepconv1 - False
18 block3_sepconv1_bn - False
19 block3_sepconv2_act - False
20 block3_sepconv2 - False
21 block3_sepconv2_bn - False
22 conv2d_1 - False
23 block3_pool - False
24 batch_normalization_1 - False
25 add_1 - False
26 block4_sepconv1_act - False
27 block4_sepconv1 - False
28 block4_sepconv1_bn - False
29 block4_sepconv2_act - False
30 block4_sepconv2 - False
31 block4_sepconv2_bn - False
32 conv2d_2 - False
33 block4_pool - False
34 batch_normalization_2 - False
35 add_2 - False
36 block5_sepconv1_act - False
37 block5_sepconv1 - False
38 bl

In [10]:
    # Add Flatten and Dense layers on top of Xception
    # Now, we need to connect our pretrained model with the new layers 
    # of our model. We can use global pooling or a flatten layer to connect 
    # the dimensions of the previous layers with the new layers. 
    
    to_res = (224, 224)
    model = K.models.Sequential()
    model.add(K.layers.Lambda(lambda image: tf.image.resize(image, to_res))) 
    model.add(Xception_Model)
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(256, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(128, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
   # model.add(K.layers.Dense(64, activation='relu'))
   # model.add(K.layers.Dropout(0.5))
   # model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(100, activation='softmax'))

In [11]:
# Compile model and train
# Results
# We obtained an accuracy of 94% on training set and 90% on validation with 10 epochs.
# In the 8th epoch, the values are very similar and it is interesting to note that 
# in the first validation accuracy is higher than training. 
# This is because of dropout use, which in Keras, it has a different behavior 
# for training and testing. In testing time, all the features are ready and 
# the dropout is turned off, resulting in a better accuracy. 
# This readjust on the last epochs since the model continues changing on the training.

model.compile(loss='categorical_crossentropy',
                  optimizer='Adadelta',
                  metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=30, epochs=6, verbose=1,
                        validation_data=(x_test, y_test)
                       )
model.summary()


Train on 50000 samples, validate on 10000 samples
Epoch 1/6
50000/50000 [==============================] - 332s 7ms/sample - loss: 5.0476 - acc: 0.0126 - val_loss: 4.5516 - val_acc: 0.0288
Epoch 2/6
50000/50000 [==============================] - 321s 6ms/sample - loss: 4.8808 - acc: 0.0187 - val_loss: 4.3532 - val_acc: 0.0603
Epoch 3/6
50000/50000 [==============================] - 321s 6ms/sample - loss: 4.7564 - acc: 0.0276 - val_loss: 4.1883 - val_acc: 0.1007
Epoch 4/6
50000/50000 [==============================] - 322s 6ms/sample - loss: 4.6430 - acc: 0.0349 - val_loss: 4.0385 - val_acc: 0.1458
Epoch 5/6
50000/50000 [==============================] - 321s 6ms/sample - loss: 4.5539 - acc: 0.0438 - val_loss: 3.9162 - val_acc: 0.1836
Epoch 6/6
50000/50000 [==============================] - 321s 6ms/sample - loss: 4.4617 - acc: 0.0526 - val_loss: 3.8002 - val_acc: 0.2149
Model: "sequential"
_________________________________________________________________
Layer (type)                 O

In [12]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 54s 5ms/sample - loss: 3.8002 - acc: 0.2149


[3.800191848373413, 0.2149]